In [ ]:
import h5py

from librosa.display import specshow
import numpy as np
import os
import pandas as pd
import tqdm

data_dir = "/beegfs/vl1019/waspaa2019_data"
ccb_dir = os.path.join(data_dir, "ccb18")
csv_name = 'CCB_2009Feb18to22Apr17.csv'
csv_path = os.path.join(ccb_dir, csv_name)

hop_length = 2**7
n_fft = 2 * hop_length
sr = 2000
half_clip_n_cols = 16
low_freq_bin = 5

pcen_version_id = 2
pcen_version_str = str(pcen_version_id)
hdf5_dir = os.path.join(
    ccb_dir, "ccb18_h5_v-" + pcen_version_str)
distances = []
dates = []
channels = []

stft_max_list = []
stft_avg_list = []
log_pcen_max_list = []
log_pcen_avg_list = []


df = pd.read_csv(csv_path)
df = df.dropna()
df = df.sort_values(by="DistanceKm")


import h5py

from librosa.display import specshow
import numpy as np
import os
import pandas as pd
import tqdm

data_dir = "/beegfs/vl1019/waspaa2019_data"
ccb_dir = os.path.join(data_dir, "ccb18")
csv_name = 'CCB_2009Feb18to22Apr17.csv'
csv_path = os.path.join(ccb_dir, csv_name)

hop_length = 2**7
n_fft = 2 * hop_length
sr = 2000
half_clip_n_cols = 16
low_freq_bin = 5

pcen_version_id = 2
pcen_version_str = str(pcen_version_id)
hdf5_dir = os.path.join(
    ccb_dir, "ccb18_h5_v-" + pcen_version_str)
distances = []
dates = []
channels = []

stft_max_list = []
stft_avg_list = []
log_pcen_max_list = []
log_pcen_avg_list = []


df = pd.read_csv(csv_path)
df = df.dropna()
df = df.sort_values(by="DistanceKm")

for i, row_id in tqdm.tqdm(enumerate(range(len(df)))):
    row = df.iloc[row_id]
    
    # Read date.
    date_int = int(row["Date"])
    date_str = "20090" + str(date_int)
    prefix_str = "CCB18_" + date_str

    # Read channel.
    channel_id = int(row["Channel"])
    channel_str = str(channel_id)

    # Read onset time
    onset_time_int = int(row["Begin.Time..s."])
    hour_str = str(onset_time_int // 3600).zfill(2)
    minute_str = str(((onset_time_int % 3600) // (60*15)) * 15).zfill(2)
    second_str = "00"
    time_str = "".join([hour_str, minute_str, second_str])
    
    # Locate HDF5 
    hdf5_name = "_".join([
        "CCB18",
        date_str,
        time_str,
        pcen_version_str
    ])
    hdf5_path = os.path.join(hdf5_dir, hdf5_name + ".h5")
    
    if not os.path.exists(hdf5_path):
        continue
        
    distances.append(row["DistanceKm"])
    dates.append(row["Date"])
    channels.append(row["Channel"])
    
    # Open HDF5
    with h5py.File(hdf5_path, 'r') as f:
        mid_time = (0.5 * (row["Begin.Time..s."] + row["End.Time..s."])) % (15*60)
        mid_col = int(round(mid_time * (sr/hop_length)))
        start_col = max(0, mid_col - half_clip_n_cols)
        stop_col = min(start_col + 2*half_clip_n_cols, f["stft"].shape[1])
        start_col = stop_col - 2*half_clip_n_cols
        X_stft = f["stft"][low_freq_bin:, start_col:stop_col, channel_id-1]
        X_stft_sf = np.maximum(0, np.diff(np.log1p(X_stft), axis=1))
        stft_max_list.append(np.max(X_stft_sf))
        stft_avg_list.append(np.max(np.mean(X_stft_sf, axis=0)))
        X_pcen = f["pcen"][low_freq_bin:, start_col:stop_col, channel_id-1]
        log_pcen_max_list.append(np.max(np.log1p(X_pcen)))
        log_pcen_avg_list.append(np.max(np.mean(np.log1p(X_pcen), axis=0)))

feature_dict = {
    "stft_avg": stft_avg_list,
    "stft_max": stft_max_list,
    "log_pcen_avg": log_pcen_avg_list,
    "log_pcen_max": log_pcen_max_list,
    "distance": distances,
    "date": dates,
    "channel": channels
}
feature_df = pd.DataFrame(feature_dict)

feature_df.to_csv(
    "/beegfs/vl1019/waspaa2019_data/ccb18/ccb18_features_v-2bis" +\
    pcen_version_str + ".csv")

In [ ]:
import glob
import librosa
import soundfile as sf

ships_dir = '/beegfs/vl1019/waspaa2019_data/shipsEar_AUDIOS'
settings = {
    "T": 1.0,
    "alpha": 1.0,
    "delta": 0.0,
    "r": 1.0,
    "eps": 1e-6,
    "n_fft": 2**8,
    "win_length": 2**8,
    "hop_length": 2**7,
    "sr": 2000
}

stft_avg_neg = []
stft_max_neg = []
log_pcen_avg_neg = []
log_pcen_max_neg = []


for ship_path in tqdm.tqdm(glob.glob(os.path.join(ships_dir, "*.wav"))):

    ship_waveform, orig_sr = sf.read(ship_path)

    ship_waveform = librosa.resample(ship_waveform, orig_sr, settings["sr"])

    stft = librosa.stft(
        ship_waveform * (2**31),
        n_fft=settings["n_fft"],
        win_length=settings["win_length"],
        hop_length=settings["hop_length"],
        window="hann")[low_freq_bin:, :]

    logspec = np.log1p(np.abs(stft))

    logspec_flux = np.maximum(0, np.diff(logspec, axis=1))
    stft_avg_neg.append(np.mean(logspec_flux, axis=0))
    stft_max_neg.append(np.max(logspec_flux, axis=0))

    pcen = librosa.pcen(np.abs(stft),
        sr=settings["sr"],
        hop_length=settings["hop_length"],
        gain=settings["alpha"],
        bias=settings["delta"],
        power=settings["r"],
        time_constant=settings["T"],
        eps=settings["eps"])[low_freq_bin:, :]

    log_pcen_avg_neg.append(np.mean(np.log1p(pcen), axis=0))
    log_pcen_max_neg.append(np.max(np.log1p(pcen), axis=0))
    
    
neg_dict = {
    "log_pcen_avg": np.concatenate([x[10:-10] for x in log_pcen_avg_neg]),
    "log_pcen_max": np.concatenate([x[10:-10] for x in log_pcen_max_neg]),
    "stft_avg": np.concatenate([x[10:-10] for x in stft_avg_neg]),
    "stft_max": np.concatenate([x[10:-10] for x in stft_max_neg])
}

In [ ]:
with h5py.File('/beegfs/vl1019/waspaa2019_data/ccb18/ccb18_negatives_v-' + pcen_version_str + "bis.hdf5", 'w') as f:
    for k in neg_dict.keys():
        f[k] = neg_dict[k]

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

neg_dict = {}

with h5py.File('/beegfs/vl1019/waspaa2019_data/ccb18/ccb18_negatives_v-' + pcen_version_str + "bis.hdf5", 'r') as f:
    for k in f.keys():
        neg_dict[k] = f[k][()]
        
        
feature_df = pd.DataFrame.from_csv(
    "/beegfs/vl1019/waspaa2019_data/ccb18/ccb18_features_v-2bis" +\
    pcen_version_str + ".csv")

max_distance = 21
command_tpr = 0.5
dates = [219, 220, 221, 222, 417]
feature_mtbfs = []

feature_strs = ["log_pcen_max", "stft_avg", "stft_max", "log_pcen_avg"]

for feature_str in feature_strs:

    distance_mtbfs = []

    for min_distance in range(30):
        dist_rows =\
                (feature_df["distance"] >= min_distance) &\
                (feature_df["distance"] < (min_distance+1)) &\
                (feature_df["channel"] != 3)

        mtbfs = []
        for date in dates:
            date_dist_rows = dist_rows & (feature_df["date"] == date)
            sorted_values = np.sort(feature_df[date_dist_rows][feature_str])[::-1]
            n_positives = len(sorted_values)
            command_threshold = sorted_values[int(command_tpr*n_positives)]
            command_fpr = np.mean(neg_dict[feature_str] > command_threshold)
            mtbf = (hop_length/sr) / command_fpr
            mtbfs.append(mtbf)

        distance_mtbfs.append(np.stack(mtbfs))
        
    feature_mtbfs.append(np.stack(distance_mtbfs))
    
feature_mtbfs = np.array(feature_mtbfs)

plt.figure(figsize=(6, 4))
plt.title('Bioacoustic detection of North Atlantic Right Whale', size=12)

plt.plot(
    np.log10(np.median(feature_mtbfs[0, :max_distance, :], axis=1)),
    '-o', markersize=10.0, linewidth=2.0, color="#009900",
    label="Max-pooled PCEN")
plt.fill_between(
    range(max_distance),
    np.log10(np.ravel(np.quantile(feature_mtbfs[0, :max_distance, :], [0.25], axis=1))),
    np.log10(np.ravel(np.quantile(feature_mtbfs[0, :max_distance, :], [0.75], axis=1))),
    alpha = 0.5,
    color="#009900",
)


plt.plot(
    np.log10(np.median(feature_mtbfs[2, :max_distance, :], axis=1)),
    '-s', color="#0000B2",
    markersize=10.0, linewidth=2.0,
    label="Max-pooled spectral flux")
plt.fill_between(
    range(max_distance),
    np.log10(np.ravel(np.quantile(feature_mtbfs[2, :max_distance, :], [0.25], axis=1))),
    np.log10(np.ravel(np.quantile(feature_mtbfs[2, :max_distance, :], [0.75], axis=1))),
    alpha = 0.5,
    color="#0000B2"
)


plt.plot(
    np.log10(np.median(feature_mtbfs[1, :max_distance, :], axis=1)), '-v', markersize=10.0, linewidth=2.0,
    color="#E67300", label="Averaged spectral flux")
plt.fill_between(
    range(max_distance),
    np.log10(np.ravel(np.quantile(feature_mtbfs[1, :max_distance, :], [0.25], axis=1))),
    np.log10(np.ravel(np.quantile(feature_mtbfs[1, :max_distance, :], [0.75], axis=1))),
    alpha = 0.33, color="#E67300"
)

yticks = np.array([0.5, 1.0, 2.0, 5.0, 10.0, 20.0, 50.0, 100.0, 200.0])


plt.rcParams["font.family"] = "serif"
plt.legend(prop={'size': 11})
plt.xlabel("CCB18 dataset: distance between sensor and source (km)", size=12)
plt.ylabel("ShipsEar dataset: mean time\nbetween false alarms at half recall (s)", size=12)
plt.gca().set_xticks(np.arange(0, 21, 2), minor=True)
plt.gca().set_xticks(np.arange(0, 21, 4), minor=False)
plt.gca().set_xticklabels(np.arange(0, 21, 4), size=11)
plt.yticks(np.log10(yticks))
plt.gca().set_yticklabels(yticks, size=11)
plt.xlim(0, 20)
plt.grid(linestyle='--', alpha=1.0, which="minor")
plt.grid(linestyle='--', alpha=1.0, which="major")
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.savefig('lostanlen_waspaa2019_ccb_mtbfa-50_semilogy.eps', bbox_inches='tight')
plt.savefig('lostanlen_waspaa2019_ccb_mtbfa-50_semilogy.png', bbox_inches='tight', dpi=500)